In [1]:
import numpy as np
import pandas as pd
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from collections import Counter
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [2]:
# read the dataset
reviews_data = pd.read_json(r"C:\Users\user\Documents\UoM\Thesis\data\Clothing_Shoes_and_Jewelry_5.json", lines=True)

# Keep only the review text and the grade
reviews_data = reviews_data[['overall', 'reviewText']]


# Drop the products whose values are null
reviews_data = reviews_data[reviews_data['reviewText'].notna()]

In [3]:
ratings = []
for index,entry in enumerate(reviews_data['overall']):
    if entry == 1.0 or entry == 2.0:
        ratings.append(-1)
    elif entry == 3.0:
        ratings.append(0)
    elif entry == 4.0 or entry == 5.0:
        ratings.append(1)

In [4]:
""""
Split each review into sentences
and preprocess each sentence
"""
stopwords = stopwords.words('english')

preprocessed_data = []
index = 0
for review in reviews_data['reviewText']:
    if(index % 10000 == 0):
        print(index)
    review_sentences = tokenizer.tokenize(review)
    for sentence in review_sentences:
        if(sentence != ''):
            review = {}
            review["index"] = index
            review["sentence"] = sentence
            review["rating"] = ratings[index]
            preprocessed_data.append(review)
    index += 1

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000


In [5]:
print("Before preprocessing: ", reviews_data['reviewText'][278643])
print("------------------------------------------------")
print("After preprocessing: ", [d for d in preprocessed_data if d['index'] == 278643])
print(len(preprocessed_data))

Before preprocessing:  idk why this shirt didnt fit i mean i ordered an Xlits very short and it seems like it was cut weird because the cross sits funny, like sideways.also one of the studs fell off the second i tried it on. ahhhhhhh whatever thats what i get for ordering clothes online loli DO NOT recommend
------------------------------------------------
After preprocessing:  [{'index': 278643, 'sentence': 'idk why this shirt didnt fit i mean i ordered an Xlits very short and it seems like it was cut weird because the cross sits funny, like sideways.also one of the studs fell off the second i tried it on.', 'rating': -1}, {'index': 278643, 'sentence': 'ahhhhhhh whatever thats what i get for ordering clothes online loli DO NOT recommend', 'rating': -1}]
1185548


In [6]:
df = pd.DataFrame(preprocessed_data)

In [7]:
def calculate_textblob_score(sentence):
    return TextBlob(sentence).sentiment.polarity

In [8]:
review = [d for d in preprocessed_data if d['index'] == 234596]
print(review)
for sentence in review:
    print(calculate_textblob_score(sentence['sentence']))
print(calculate_textblob_score(reviews_data['reviewText'][234596]))

[{'index': 234596, 'sentence': 'Great hat.', 'rating': 1}, {'index': 234596, 'sentence': 'Fit is right on and looks just too cute on.', 'rating': 1}, {'index': 234596, 'sentence': "I don't really know what else to say, it is a hat and it is cute but I had not fulfilled the minimum number of words yet at the start of this sentence.", 'rating': 1}]
0.8
0.3952380952380952
0.35
0.44761904761904764


In [9]:
def get_sentiment_from_score(predicted_score):
    if predicted_score > 0:
        return 1
    elif predicted_score < -0:
        return -1
    else:
        return 0

In [10]:
# Calculate predicted scores using afinn lexicon approcach
df['predicted_score'] = df.apply(lambda x: calculate_textblob_score(x['sentence']), axis=1)

# Get the sentiment from the caluclated score
df['predicted_sentiment'] = df.apply(lambda x: get_sentiment_from_score(x['predicted_score']), axis=1)

In [11]:
df.head()

,index,rating,sentence,predicted_score,predicted_sentiment
0,0,1,This is a great tutu and at a really great price.,0.800000,1
1,0,1,It doesn't look cheap at all.,0.400000,1
2,0,1,I'm so glad I looked on Amazon and found such ...,0.033333,1
3,0,1,A++,0.000000,0
4,1,1,I bought this for my 4 yr old daughter for dan...,0.283333,1


In [12]:
def find_Max_Element(scores):
    c = Counter(scores)
    negatives = c[-1]
    neutral = c[0]
    positives = c[1]

    if(neutral == positives and neutral > negatives and positives > negatives):
        max_element = 1
    elif (neutral == negatives and neutral > positives and negatives > positives):
        max_element = -1
    elif (neutral == negatives and negatives == positives):
        max_element = 0
    elif (positives == negatives and positives > neutral and negatives > neutral) :
        max_element = 0
    else:
        max_element = max(set(scores), key = scores.count)
        
    return max_element

In [13]:
# Report 

# for each review get all scores by review
scores_by_review_frame = df.groupby('index')['predicted_sentiment'].apply(list)
majority_scores =  scores_by_review_frame.apply(find_Max_Element)
predicted_scores = list(majority_scores)

# for each review get its actual score
actual_scores_frame = df.groupby('index')['rating'].first()
actual_scores = list(actual_scores_frame)

# Calculate Accuracy
accuracy = accuracy_score(predicted_scores, actual_scores)*100
print("Accuracy: ", accuracy)

# Confusion Matrix
cm = confusion_matrix(actual_scores, predicted_scores)
print("Confusion matrix: ", cm)

# Classification Report
my_tags = ['Negative','Neutral','Positive']
report = classification_report(actual_scores, predicted_scores, target_names=my_tags);
print(report)

Accuracy:  70.25081373608035
Confusion matrix:  [[  6437   8549  11666]
 [  4006   7407  19010]
 [  7738  31928 181912]]
              precision    recall  f1-score   support

    Negative       0.35      0.24      0.29     26652
     Neutral       0.15      0.24      0.19     30423
    Positive       0.86      0.82      0.84    221578

    accuracy                           0.70    278653
   macro avg       0.45      0.44      0.44    278653
weighted avg       0.73      0.70      0.71    278653

